In [9]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam, AdamW
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [30]:
# 모델 인스턴스 생성
model_name = 'tf_efficientnet_b5.ns_jft_in1k'#'tiny_vit_21m_384.dist_in22k_ft_in1k'#'efficientnetv2_rw_m.agc_in1k' #'vit_base_patch16_224.augreg_in1k' #'efficientnet_b3.ra2_in1k'#'densenet121.ra_in1k'# #'resnet101' #'resnet34' # 'resnet50' 'efficientnet-b0', ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# training config
pre_img_size = 600
img_size = 456 #224 #256
LR = 5e-4
EPOCHS = 5
BATCH_SIZE = 8
num_workers = 4
early_stopping_patience = 5  # Early Stopping 설정
augment_ratio = 200
num_classes=17

model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17,
    drop_rate=0.2
).to(device)

# 가중치 로드
model.load_state_dict(torch.load(r'/data/ephemeral/home/notebook/tf_efficientnet_b5.ns_jft_in1k_epoch_4.pt'))


<All keys matched successfully>

In [31]:
# https://demo.albumentations.ai/
import albumentations as A
from albumentations.pytorch import ToTensorV2

# horizontal_flip, vertical_flip, double_flip, transpose 등 변환 정의
horizontal_flip = A.HorizontalFlip(p=1)
vertical_flip = A.VerticalFlip(p=1)
double_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])
transpose = A.Transpose(p=1)
transpose_hflip = A.Compose([
    A.Transpose(p=1), 
    A.HorizontalFlip(p=1),
])
transpose_vflip = A.Compose([
    A.Transpose(p=1),
    A.VerticalFlip(p=1),
])
transpose_dflip = A.Compose([
    A.Transpose(p=1),  
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
])

# Augmentation을 위한 transform 코드
trn_transform = A.Compose([
    A.LongestMaxSize(max_size=pre_img_size, always_apply=True),
    A.PadIfNeeded(min_height=pre_img_size, min_width=pre_img_size, border_mode=0, value=(255, 255, 255)),
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 800.0), p=1),
        A.GaussianBlur(blur_limit=(1, 7), p=1),
        A.MotionBlur(blur_limit=(3, 7), p=1),
        A.MedianBlur(blur_limit=3, p=1)
    ], p=1),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.25),
    A.Rotate(limit=(0, 360), p=0.75),
    A.GridDistortion(always_apply=False, p=0.75, num_steps=6, distort_limit=(-0.3, 0.3), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None, normalized=False),
    A.OneOf([
        horizontal_flip,
        vertical_flip,
        double_flip,
        transpose,
        transpose_hflip,
        transpose_vflip,
        transpose_dflip
    ], p=1.0),
    A.OneOf([
        A.OpticalDistortion(always_apply=False, p=1.0, distort_limit=(-0.3, 0.3), shift_limit=(-0.05, 0.09), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
        A.MultiplicativeNoise(always_apply=False, p=1.0, multiplier=(0.93, 2.22), per_channel=True, elementwise=True),
        A.ISONoise(always_apply=False, p=1.0, intensity=(0.38, 1.0), color_shift=(0.18, 0.47)),
        A.RandomBrightnessContrast(always_apply=False, p=1.0, brightness_limit=(0.19, 0.62), contrast_limit=(-0.02, 0.62), brightness_by_max=True)
    ], p=0.75),
    
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    #Adjust(always_apply=True),
    A.LongestMaxSize(max_size=pre_img_size, always_apply=True),
    A.PadIfNeeded(min_height=pre_img_size, min_width=pre_img_size, border_mode=0, value=(255, 255, 255)),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])


In [32]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None, oversample=False, augment_ratio=1):
        self.df = pd.read_csv(csv)
        self.path = path
        self.transform = transform
        self.oversample = oversample
        self.augment_ratio = augment_ratio

        # 클래스간 불균형 해소를 위한 샘플 증식
        if self.oversample:
            # 각 클래스별로 데이터 수 계산
            class_counts = np.bincount(self.df.values[:, 1].astype(int))

            # 각 클래스별로 증식할 횟수 설정 (이 예제에서는 최대 데이터 수에 맞춤)
            max_class_count = max(class_counts)
            oversample_factors = [max_class_count // count for count in class_counts]
            # Class 3, 7 가중치 2로 변경
            # oversample_factors[3] = 2
            # oversample_factors[7] = 2 
            # oversample_factors[14] = 3 

            # 각 클래스별로 데이터를 증식한 새로운 데이터 프레임 생성
            oversampled_data = [self.df.values[self.df.values[:, 1] == cls].repeat(factor, axis=0) for cls, factor in enumerate(oversample_factors)]
            oversampled_data = np.vstack(oversampled_data)

            self.df = pd.DataFrame(oversampled_data, columns=self.df.columns)

    def __len__(self):
        return len(self.df) * self.augment_ratio

    def __getitem__(self, idx):
        real_idx = idx % len(self.df)
        name, target = self.df.iloc[real_idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target
    
    
# meta.csv 파일 읽기
meta_data = pd.read_csv('/data/ephemeral/home/datasets_fin/meta.csv')
label_to_class_name = dict(zip(meta_data['target'], meta_data['class_name']))

In [33]:
# 데이터셋 및 데이터 로더 정의
trn_dataset = ImageDataset(
    "/data/ephemeral/home/datasets_fin/train_labelupdate.csv",
    "/data/ephemeral/home/datasets_fin/combined_train/train",
    transform=trn_transform,
    oversample=True,
    augment_ratio=augment_ratio
)


tst_dataset = ImageDataset(
    "/data/ephemeral/home/datasets_fin/sample_submission.csv",
    "/data/ephemeral/home/datasets_fin/wdnx4/wdnx4",
    transform=tst_transform,
    oversample=False,
    augment_ratio=1
)


ori_traindata_num = int(len(trn_dataset)/augment_ratio)
print(f"Original training data count: {ori_traindata_num}")
print(f"Augmented training data count: {len(trn_dataset)}")
print(f"Test data count: {len(tst_dataset)}")


Original training data count: 1666
Augmented training data count: 333200
Test data count: 3140


In [34]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)



In [35]:
import pandas as pd
import torch
from torch.nn.functional import softmax
from tqdm import tqdm

# 모델 평가 모드 설정
model.eval()
preds_list = []
probs_list = []  # 확률을 저장할 리스트

# DataLoader를 통해 이미지를 반복 처리
for image, _ in tqdm(tst_loader):
    image = image.to(device)
    with torch.no_grad():
        preds = model(image)
        probs = softmax(preds, dim=1)  # Softmax를 적용하여 확률 계산
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    probs_list.extend(probs.detach().cpu().numpy())  # 확률 값 저장

# 예측 결과와 확률을 데이터프레임으로 저장
pred_df = pd.DataFrame({
    'ID': tst_dataset.df['ID'],
    'target': preds_list
})
# 확률을 데이터프레임에 추가
for i in range(num_classes):
    pred_df[f'prob_class_{i}'] = [probs[i] for probs in probs_list]

# 제출 형식 파일을 읽어와 ID 열이 일치하는지 확인
sample_submission_df = pd.read_csv("/data/ephemeral/home/datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

# 예측 결과와 확률을 CSV 파일로 저장
pred_df.to_csv(f"{model_name}_{img_size}SIZE_{BATCH_SIZE}BATCH_{EPOCHS}EPOCH_0810_pred.csv", index=False)
print(pred_df.head())


100%|██████████| 393/393 [03:20<00:00,  1.96it/s]

                     ID  target  prob_class_0  prob_class_1  prob_class_2  \
0  0008fdb22ddce0ce.jpg       2  3.686680e-12  3.041082e-13  9.999992e-01   
1  00091bffdffd83de.jpg      12  5.598677e-11  1.882266e-08  3.200946e-11   
2  00396fbc1f6cc21d.jpg       5  3.481447e-15  5.992583e-17  1.695421e-15   
3  00471f8038d9c4b6.jpg      12  4.267822e-09  1.425181e-07  4.010682e-07   
4  00901f504008d884.jpg       2  2.306215e-11  4.956328e-17  1.000000e+00   

   prob_class_3  prob_class_4  prob_class_5  prob_class_6  prob_class_7  \
0  3.284950e-07  1.482153e-13  7.416641e-11  1.066299e-10  1.534541e-12   
1  5.146481e-11  1.454109e-11  2.926739e-11  1.333111e-07  1.134807e-12   
2  5.550253e-13  3.708128e-16  1.000000e+00  4.406705e-16  5.967123e-14   
3  3.882549e-06  6.846597e-06  3.906863e-08  2.736593e-06  2.220130e-05   
4  1.360666e-10  1.746026e-12  5.230599e-11  4.736079e-16  5.024436e-11   

   prob_class_8  prob_class_9  prob_class_10  prob_class_11  prob_class_12  \
0  3.046

In [24]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from PIL import Image, ImageEnhance
import pandas as pd
import os
import cv2
from torch.nn.functional import softmax

# Albumentations transform 정의
pre_img_size = 512  # 사전 정의된 이미지 크기 (필요에 따라 수정)

transform = A.Compose([
    A.LongestMaxSize(max_size=pre_img_size, always_apply=True),
    A.PadIfNeeded(min_height=pre_img_size, min_width=pre_img_size, border_mode=0, value=(255, 255, 255)),
    # A.Resize(height=pre_img_size, width=pre_img_size),  # 모든 이미지를 같은 크기로 리사이징
    #A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 여기서 normalize넣으면 완전히 잘못됨 후에 또 하기 때문
    ToTensorV2(),
])


class ImageDataset2(Dataset):
    def __init__(self, csv_file, path, transform=None):
        self.df = pd.read_csv(csv_file)
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        real_idx = idx % len(self.df)
        name, target = self.df.iloc[real_idx]
        img_path = os.path.join(self.path, name)
        
        try:
            img = np.array(Image.open(img_path).convert("RGB"))
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            img = np.zeros((pre_img_size, pre_img_size, 3), dtype=np.uint8)  # 빈 이미지로 대체
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# TTA 변환 정의
tta_transforms = [
    A.Compose([
        # A.GaussNoise(var_limit=(10.0, 800.0), p=0.75),
        # A.GaussianBlur(blur_limit=(1, 7), p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    A.Compose([
        # A.GaussNoise(var_limit=(10.0, 800.0), p=0.75),
        # A.GaussianBlur(blur_limit=(1, 7), p=0.5),
        A.HorizontalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    A.Compose([
        # A.GaussNoise(var_limit=(10.0, 800.0), p=0.75),
        # A.GaussianBlur(blur_limit=(1, 7), p=0.5),
        A.VerticalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    A.Compose([
        # A.GaussNoise(var_limit=(10.0, 800.0), p=0.75),
        # A.GaussianBlur(blur_limit=(1, 7), p=0.5),
        A.Transpose(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    A.Compose([
        A.Transpose(p=1.0),
        A.HorizontalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    A.Compose([
        # A.GaussNoise(var_limit=(10.0, 800.0), p=0.75),
        # A.GaussianBlur(blur_limit=(1, 7), p=0.5),
        A.Transpose(p=1.0),
        A.VerticalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    A.Compose([
        # A.GaussNoise(var_limit=(10.0, 800.0), p=0.75),
        # A.GaussianBlur(blur_limit=(1, 7), p=0.5),
        A.Transpose(p=1.0),
        A.HorizontalFlip(p=1.0),
        A.VerticalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]),
    # A.Compose([
    #     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #     ToTensorV2()
    # ])  # 원본 이미지
]

tta_dataset = ImageDataset2(
    "/data/ephemeral/home/datasets_fin/sample_submission.csv",
    "/data/ephemeral/home/datasets_fin/test/",
    transform=transform
)

tta_loader = DataLoader(
    tta_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

# TTA inference function
def tta_inference(loader, model, device, tta_transforms):
    model.eval()
    all_outputs = []
    
    weights = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2]  # 각 변환에 대한 가중치 설정
    
    for images, _ in tqdm(loader):
        images = images.to(device).float()  # 이미지 텐서를 float 형식으로 변환
        batch_outputs = torch.zeros(images.size(0), 17).to(device)  # 17은 클래스 수
        
        for weight, tta_transform in zip(weights, tta_transforms):
            tta_images = []
            for image in images:
                tta_image = tta_transform(image=image.permute(1, 2, 0).cpu().numpy().astype(np.float32))['image']
                tta_images.append(tta_image.to(device).float())
            tta_images = torch.stack(tta_images)
            with torch.no_grad():
                preds = model(tta_images)
                batch_outputs += weight * preds  # 가중치 적용
        
        # TTA 평균 내기 (Soft Voting)
        batch_outputs /= len(tta_transforms)
        
        # 최종 예측 값을 리스트에 저장
        all_outputs.append(batch_outputs.cpu().numpy())
    
    # 모든 배치의 예측 값을 연결
    all_outputs = np.concatenate(all_outputs, axis=0)
    return all_outputs

# TTA를 적용한 예측
all_outputs = tta_inference(tta_loader, model, device, tta_transforms)
preds_list = np.argmax(all_outputs, axis=1)

# 예측 결과를 데이터프레임으로 저장
pred_df = pd.DataFrame(tta_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

# 제출 형식 파일을 읽어와 ID 열이 일치하는지 확인
sample_submission_df = pd.read_csv("/data/ephemeral/home/datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

# 예측 결과를 CSV 파일로 저장
pred_df.to_csv(f"{model_name}_{pre_img_size}SIZE_{BATCH_SIZE}BATCH_{EPOCHS}EPOCH_0811_TTA_pred.csv", index=False)
print(pred_df.head())


100%|██████████| 393/393 [06:22<00:00,  1.03it/s]

                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      12
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg      12
4  00901f504008d884.jpg       2


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.metrics import f1_score
import matplotlib.font_manager as fm
from matplotlib.font_manager import FontProperties


def load_predictions(file_paths):
    """여러 모델의 예측 결과를 로드합니다."""
    predictions = {}
    for path in file_paths:
        model_name = os.path.basename(path).split('.')[0]
        df = pd.read_csv(path)
        # ID 컬럼 중복 제거 및 예측 컬럼 이름 변경
        df = df[['ID', 'target']].rename(columns={'target': model_name})
        predictions[model_name] = df
    return predictions

def find_different_predictions(predictions):
    """모든 모델에서 다르게 예측한 항목을 찾습니다."""
    # 모든 예측을 하나의 DataFrame으로 병합
    all_predictions = predictions[list(predictions.keys())[0]]
    for model, df in list(predictions.items())[1:]:
        all_predictions = pd.merge(all_predictions, df, on='ID', suffixes=('', f'_{model}'))
    
    # 예측 컬럼만 선택
    prediction_columns = [col for col in all_predictions.columns if col != 'ID']
    
    # 예측이 다른 행만 선택
    different_predictions = all_predictions[all_predictions[prediction_columns].nunique(axis=1) > 1]
    return different_predictions

def calculate_macro_f1(ground_truth, predictions):
    """Macro F1 점수를 계산합니다."""
    return f1_score(ground_truth, predictions, average='macro')

def plot_error_distribution(ground_truth, predictions):
    """각 클래스별 오류 예측 개수를 bar plot으로 표시합니다."""
    # 클래스 이름 정의
    class_names = {
        0: "계좌번호(손글씨)", 1: "임신출산 진료비 지급 신청서", 2: "자동차 계기판", 3: "입퇴원 확인서", 4: "진단서", 
        5: "운전면허증", 6: "진료비영수증", 7: "통원/진료 확인서", 8: "주민등록증", 9: "여권", 
        10: "진료비 납입 확인서", 11: "약제비 영수증", 12: "처방전", 13: "이력서", 14: "소견서", 
        15: "자동차 등록증", 16: "자동차 번호판"
    }

    error_counts = {i: 0 for i in range(len(class_names))}  # 모든 클래스에 대해 초기화
    for gt, pred in zip(ground_truth, predictions):
        if gt != pred:
            error_counts[gt] += 1
    
    classes = sorted(error_counts.keys())
    counts = [error_counts[c] for c in classes]
    class_labels = [f"{c}\n{class_names[c]}" for c in classes]
    
    # 나눔 폰트 설정
    # font_path = './font/NanumGothic.otf'
    # font_prop = FontProperties(fname=font_path)
    
    plt.figure(figsize=(20, 10))
    bars = plt.bar(class_labels, counts)
    plt.title("오류 예측 개수 (클래스별)", fontsize=16)# fontproperties=font_prop, 
    plt.xlabel("클래스", fontsize=14)# fontproperties=font_prop, 
    plt.ylabel("오류 개수", fontsize=14)# fontproperties=font_prop, 
    plt.xticks(rotation=45, ha='center')
    
    # x축 레이블에 폰트 적용
    ax = plt.gca()
    ax.set_xticklabels(class_labels, fontsize=10) # fontproperties=font_prop, 
    
    plt.tight_layout()
    
    # 각 막대 위에 값 표시
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height}',
                 ha='center', va='bottom')# fontproperties=font_prop, 
    
    plt.show()  
    
    
def display_images_and_predictions(different_predictions, image_dir, ground_truth_name, prediction_name, class_filter=None, max_images_per_class=3):
    """다르게 예측된 항목의 이미지와 예측 결과를 표시합니다."""
    # 나눔 폰트 설정
    # font_path = './font/NanumGothic.otf'
    # font_prop = FontProperties(fname=font_path)
    class_names = {
        0: "계좌번호(손글씨)", 1: "임신출산 진료비 지급 신청서", 2: "자동차 계기판", 3: "입퇴원 확인서", 4: "진단서", 
        5: "운전면허증", 6: "진료비영수증", 7: "통원/진료 확인서", 8: "주민등록증", 9: "여권", 
        10: "진료비 납입 확인서", 11: "약제비 영수증", 12: "처방전", 13: "이력서", 14: "소견서", 
        15: "자동차 등록증", 16: "자동차 번호판"
    }
    
    # 클래스별로 이미지 분류
    class_images = {c: [] for c in range(len(class_names))}
    for idx, row in different_predictions.iterrows():
        ground_truth = row[ground_truth_name]
        if class_filter is None or ground_truth in class_filter:
            class_images[ground_truth].append(row)
    
    # 표시할 이미지 선택
    images_to_display = []
    for c, imgs in class_images.items():
        if class_filter is None or c in class_filter:
            images_to_display.extend(imgs[:max_images_per_class])
    
    # 이미지 표시
    num_images = len(images_to_display)
    rows = (num_images - 1) // 3 + 1
    fig, axs = plt.subplots(rows, 4, figsize=(20, 5*rows))
    
    for i, row in enumerate(images_to_display):
        ax = axs[i//4, i%4] if rows > 1 else axs[i%4]
        
        image_id = row['ID']
        ground_truth = row[ground_truth_name]
        prediction = row[prediction_name]
        image_path = os.path.join(image_dir, image_id)
        
        if os.path.exists(image_path):
            img = Image.open(image_path)
            img = img.resize((300, 300), Image.LANCZOS)
            ax.imshow(img)
            ax.axis('off')
            id = f"ID: {image_id}"
            comparison_text = f"정답: {ground_truth} ({class_names[ground_truth]})\n예측: {prediction} ({class_names[prediction]})"
            ax.set_title(f"{id}\n{comparison_text}", fontsize=10) # fontproperties=font_prop, 
        else:
            ax.text(0.5, 0.5, f"Image not found\nfor ID: {image_id}", ha='center', va='center')
    
    # 빈 서브플롯 제거
    for i in range(num_images, rows*4):
        ax = axs[i//4, i%4] if rows > 1 else axs[i%4]
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"총 {num_images}개의 이미지가 표시되었습니다.")    

# 예측 파일 경로
file_paths = [
    "/data/ephemeral/home/notebook/EASY_pred_ocr_0811_9766_en14_final_submission.csv",
    # "/data/ephemeral/home/notebook/efficientnet_b2.ra_in1k_384SIZE_32BATCH_2EPOCH_100_0806_PADOS_pred.csv", # 수정해야할 부분
    "/data/ephemeral/home/notebook/tf_efficientnet_b5.ns_jft_in1k_456SIZE_8BATCH_5EPOCH_0811_pred_epoch4.csv"
]


# 이미지 디렉토리 경로
image_dir = "/data/ephemeral/home/datasets_fin/test"

# 예측 로드
predictions = load_predictions(file_paths)

# 다른 예측 찾기
different_predictions = find_different_predictions(predictions)

# # 결과 출력
# print(different_predictions)

# Macro F1 점수 계산
ground_truth_name = os.path.basename(file_paths[0]).split('.')[0]
prediction_name = os.path.basename(file_paths[-1]).split('.')[0]
ground_truth = predictions[ground_truth_name][ground_truth_name]
prediction = predictions[prediction_name][prediction_name]
macro_f1 = calculate_macro_f1(ground_truth, prediction)
print(f"Macro F1 Score: {macro_f1}")

# 오류 분포 그래프 표시
plot_error_distribution(ground_truth, prediction)


# 특정 클래스(예: 0, 1, 2)에 대해 클래스당 최대 3개씩 이미지 표시
display_images_and_predictions(different_predictions, image_dir, ground_truth_name, prediction_name, class_filter=[1,2], max_images_per_class=3)